# 二次型系统的参数估计和异常检测

二次型系统（[Quadratic form](https://en.wikipedia.org/wiki/Quadratic_form)）是只包含二次项，不包含常数和一次项的单变量或者多变量系统，例如下面分别是包含1, 2 和 3个特征变量的二次型系统：

$$ y = a x^2 $$

$$ y = a x^2 + b xy + c y^2 $$

$$ y = a x^2 + b y^2 + c z^2 + d xy + e xz + f yz $$

算子的 **输入** 是一个包含n个特征量（自变量）和1个响应变量的 dataframe，
根据算子 **参数** `label` 和 `features` 指定。

## 实例分析

创建一个包含3个特征变量和一个响应变量的 dataframe 作为算子输入:

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence

np.random.seed(1)
n = 200
x1 = np.random.uniform(-10, 10, n)
x2 = np.random.uniform(-4, 4, n)
x3 = np.random.uniform(-2, 8, n)
y = 2.89 * x1 ** 2 + 4.33 * x2 ** 2 + 6.1 * x1 * x2 + 5.9 * x2 * x3 + np.random.normal(size=n)

label = 'y'
features = 'x1,x2,x3'
data = pd.DataFrame(data=[x1, x2, x3, y]).T

输入 dataframe 如下所示：

In [2]:
print(data.head())

          0         1         2           3
0 -1.659560  3.601409  7.594343  190.073972
1  4.406490  0.453226  6.039609   85.946140
2 -9.997713  3.324851 -1.676769  101.002978
3 -3.953349  1.132530  5.093873   57.338514
4 -7.064882 -0.879938  2.650015  172.213852


定义公式生成函数，输入特征变量名称列表和向量变量名称，返回对应的二次型计算公式：

In [3]:
def build_formula(label: str, features: str) -> str:
    featlist = features.split(',')
    quads = ' + '.join(map(lambda feat: 'I(' + feat + ' ** 2)', featlist))
    ints = ' + '.join(
        map(lambda feat_pair: 'I(%s * %s)' % (feat_pair[0], feat_pair[1]),
                 combinations(featlist, 2)))
    return "%s ~ %s + %s" % (label, quads, ints)

用上面定义的特征量和响应量名称测试公式输出：

In [4]:
build_formula(label, features)

'y ~ I(x1 ** 2) + I(x2 ** 2) + I(x3 ** 2) + I(x1 * x2) + I(x1 * x3) + I(x2 * x3)'

使用上面的测试数据，结合二次型生成函数，检验计算结果：

In [5]:
res = sm.OLS.from_formula(build_formula(label, features), data=data).fit()
print(res.params)

Intercept     0.080076
I(x1 ** 2)    2.888446
I(x2 ** 2)    4.342995
I(x3 ** 2)   -0.003841
I(x1 * x2)    6.095735
I(x1 * x3)   -0.001274
I(x2 * x3)    5.910295
dtype: float64


把参数估计结果与第一个代码块中 $y$ 的表达式比较，考虑误差项的影响，可知参数估计结果正确。

## 系统评估

二次型系统评估的目标是找出系统中的异常值。
算子的 **输入** 与二次型参数估计相同，**参数** 除了二次型估计的参数 *lable* 和 *features* 外，还有异常判断阈值 *lt*，默认值为3。
**输出** 是在输入 dataframe 上增加一列作为异常判断标记，`True` 表示超过阈值的异常值，`False` 表示非异常值。

一个观测的 [studentized residual](https://en.wikipedia.org/wiki/Studentized_residual) 值标示了它的异常程度，大于阈值的被作为异常值标记出来。下面使用 `student_resid` 参数给出模型中每个观测的 studentized residual，并筛选出大于阈值 `threshold` 的观测：

In [6]:
rst = OLSInfluence(res).summary_frame().student_resid
rst.head()

0    0.705898
1    0.699350
2   -0.191339
3   -0.181196
4    0.480180
Name: student_resid, dtype: float64

筛选出异常值，追加到 data 的 *outlier* 列中，得到算子输出：

In [7]:
lt = 3
data['outlier'] = rst >= lt
data.head()

,0,1,2,3,outlier
0,-1.659560,3.601409,7.594343,190.073972,False
1,4.406490,0.453226,6.039609,85.946140,False
2,-9.997713,3.324851,-1.676769,101.002978,False
3,-3.953349,1.132530,5.093873,57.338514,False
4,-7.064882,-0.879938,2.650015,172.213852,False


列出异常值：

In [8]:
data[data['outlier']]

,0,1,2,3,outlier
103,-2.854605,0.555955,-0.252341,18.338275,True
